In [41]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
#convert stata data to pandas
stata_data='../NSDUH_2017.DTA'
df=pd.read_stata(stata_data)

In [47]:
df.shape[0]

56276

In [54]:
dfa = df[df['CATAG2']!='1 - 12-17 Years Old']
dfa = dfa[df.suicplan != '85 - BAD DATA Logically assigned']
dfa = dfa[df.suicplan != '94 - DON T KNOW']
dfa = dfa[df.suicplan != '97 - REFUSED']
dfa = dfa[df.suicplan != '98 - BLANK']

/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [55]:
dfa.shape[0]

42237

In [56]:
dfa.groupby(['suicplan']).size()

suicplan
1 - Yes                               845
2 - No                               1740
85 - BAD DATA Logically assigned        0
94 - DON T KNOW                         0
97 - REFUSED                            0
98 - BLANK                              0
99 - LEGITIMATE SKIP                39652
dtype: int64

In [57]:
dfa.loc[df.suicplan=='1 - Yes', 'suicide'] = 'plan'
dfa.loc[df.suicplan=='2 - No', 'suicide'] = 'thought'
dfa.loc[df.suicplan=='99 - LEGITIMATE SKIP', 'suicide'] = 'no'

In [58]:
dfa.groupby('irpnrnmrec').size()

irpnrnmrec
1 - Within the past 30 days                               648
2 - More than 30 days ago but within the past 12 mos     1596
3 - More than 12 months ago                              2953
9 - NEVER MISUSED PAIN RELIEVERS                        37040
dtype: int64

In [59]:
dfa.loc[dfa.irpnrnmrec=='1 - Within the past 30 days', 'misuse'] = 'last year'
dfa.loc[dfa.irpnrnmrec=='2 - More than 30 days ago but within the past 12 mos', 'misuse'] = 'last year'
dfa.loc[dfa.irpnrnmrec=='3 - More than 12 months ago', 'misuse'] = 'sometime'
dfa.loc[dfa.irpnrnmrec=='9 - NEVER MISUSED PAIN RELIEVERS', 'misuse'] = 'never'

In [74]:
dfb=dfa[dfa.misuse!='sometime']
dfb.shape[0]

39284

In [76]:
gf=dfa.groupby(['suicide','misuse']).size()

In [77]:
gf.unstack()[['never','sometime','last year']]

misuse,never,sometime,last year
suicide,,,
no,35182,2642,1828
plan,563,104,178
thought,1295,207,238


In [81]:
y=dfa.suicide
dummies = pd.get_dummies(y)
y=1-dummies['no']

In [82]:
X=dfa.misuse
X = 1-pd.get_dummies(X)['never']

In [84]:
dfa['s_yes_no']=dfa['suicide'].apply(lambda x: 0 if x=='no' else 1)
dfa['o_yes_no']=dfa['misuse'].apply(lambda x: 0 if x=='never' else 1)

In [85]:
dfa[['o_yes_no','s_yes_no']].corr()

,o_yes_no,s_yes_no
o_yes_no,1.000000,0.122962
s_yes_no,0.122962,1.000000


In [86]:
X=dfa['o_yes_no']
X=np.array(X).reshape(-1,1)
y=dfa['s_yes_no']

In [87]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
...                          multi_class='multinomial').fit(X, y)

In [88]:
coef=clf.coef_
coef

array([[0.56180061]])

In [89]:
np.exp(coef)

array([[1.75382761]])

p/1-p=1.7
=> p=.61